<a href="https://colab.research.google.com/github/SankethHanasi/BIS_LAB/blob/main/Lab_Cuckoo_Search_Application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#1BM22CS242 Cuckoo Search Applications
import numpy as np

# --- Supply Chain Specific Objective Function ---
def supply_chain_objective(x, demand, transport_costs, holding_costs, lead_times, capacity):

  num_locations = len(demand)

  # Example reshaping. Adjust according to your decision variables.
  # We assume x = [shipments, inventory]
  num_shipments = num_locations * num_locations  # shipments between any two locations
  shipments = x[:num_shipments].reshape(num_locations, num_locations)
  inventory = x[num_shipments:]

  total_transport_cost = 0
  total_holding_cost = 0

  # Calculate transport cost
  for i in range(num_locations):
    for j in range(num_locations):
      total_transport_cost += shipments[i, j] * transport_costs[i, j]

  # Calculate holding cost
  for i in range(num_locations):
    total_holding_cost += inventory[i] * holding_costs[i]

  # Calculate lead time cost (simplified, could be more complex)
  total_lead_time_cost = np.sum(np.sum(shipments * lead_times))

  # Capacity constraint penalty
  total_capacity_penalty = 0
  for i in range(num_locations):
    if np.sum(shipments[:,i])> capacity[i]:
      total_capacity_penalty+= 10000*(np.sum(shipments[:,i])- capacity[i])

  # Demand fulfillment constraint penalty
  total_demand_penalty = 0
  for i in range(num_locations):
    inflow = np.sum(shipments[:, i])
    if inflow < demand[i]:
        total_demand_penalty += 10000 * (demand[i] - inflow)

  total_cost = (
      total_transport_cost
      + total_holding_cost
      + total_lead_time_cost
      + total_capacity_penalty
      + total_demand_penalty
  )
  return total_cost

# --- Lévy Flight & Initialization ---
def levy_flight(D, alpha=1.0):
  u = np.random.normal(0, 1, D)
  v = np.random.normal(0, 1, D)
  step = u / (np.abs(v) ** (1 / alpha))
  return step

def randomly_initialize(N, D, lower_bound, upper_bound):
  return np.random.uniform(lower_bound, upper_bound, (N, D))

# --- Fitness Evaluation ---
def evaluate_fitness(nests, func, *args):
    return np.array([func(nest, *args) for nest in nests])

def min_fitness_index(fitness):
  return np.argmin(fitness)

# --- Nest Replacement ---
def get_worst_nests(fitness, pa, N):
  sorted_indices = np.argsort(fitness)
  worst_indices = sorted_indices[-int(pa * N):]
  return worst_indices

def replace_worst_nests(nests, worst_nests, N, D, lower_bound, upper_bound):
  for i in worst_nests:
    nests[i] = np.random.uniform(lower_bound, upper_bound, D)
  return nests

# --- Cuckoo Search Algorithm ---
def cuckoo_search(Func, D, N, MaxIter, pa, alpha, lower_bound, upper_bound, *args):
  nests = randomly_initialize(N, D, lower_bound, upper_bound)
  fitness = evaluate_fitness(nests, Func, *args)
  best_nest = nests[min_fitness_index(fitness)]
  best_fitness = np.min(fitness)

  for iteration in range(MaxIter):
    for i in range(N):
      step_size = alpha * levy_flight(D)
      new_nest = nests[i] + step_size
      new_nest = np.clip(new_nest, lower_bound, upper_bound)
      new_fitness = Func(new_nest, *args)

      if new_fitness < fitness[i]:
        nests[i] = new_nest
        fitness[i] = new_fitness
    worst_nests = get_worst_nests(fitness, pa, N)
    nests = replace_worst_nests(nests, worst_nests, N, D, lower_bound, upper_bound)

    current_best_index = min_fitness_index(fitness)
    if fitness[current_best_index] < best_fitness:
      best_nest = nests[current_best_index]
      best_fitness = fitness[current_best_index]

  return best_nest, best_fitness

# --- Example Usage ---
if __name__ == "__main__":
  # --- Supply Chain Parameters ---
  num_locations = 3
  demand = np.array([100, 150, 200])  # Demand at each location
  transport_costs = np.array([[0, 2, 3], [2, 0, 1], [3, 1, 0]])  # Transport costs between locations
  holding_costs = np.array([1, 1.5, 2])  # Holding costs at each location
  lead_times = np.array([[0, 2, 3], [2, 0, 1], [3, 1, 0]])  # Lead times between locations
  capacity = np.array([300,300,300]) # Capacities at each location

  # --- Optimization Parameters ---
  D = num_locations * num_locations + num_locations  # shipments + inventory levels
  N = 30  # Number of nests
  MaxIter = 150  # Maximum number of iterations
  pa = 0.25  # Probability of nest abandonment
  alpha = 1.0  # Scaling factor for Lévy flight
  lower_bound = 0  # Lower bound (e.g., no negative shipments)
  upper_bound = 300  # Upper bound (e.g., maximum possible shipment/inventory)

  # --- Run Cuckoo Search ---
  best_nest, best_fitness = cuckoo_search(
      supply_chain_objective,
      D,
      N,
      MaxIter,
      pa,
      alpha,
      lower_bound,
      upper_bound,
      demand,
      transport_costs,
      holding_costs,
      lead_times,
      capacity
  )

  print("Best solution (shipments and inventory):", best_nest)
  print("Best fitness (total cost):", best_fitness)

  num_shipments = num_locations * num_locations
  shipments = best_nest[:num_shipments].reshape(num_locations, num_locations)
  inventory = best_nest[num_shipments:]

  print("\nShipments Matrix:")
  print(np.round(shipments))

  print("\nInventory Levels:")
  print(np.round(inventory))


Best solution (shipments and inventory): [244.84141921  37.35373563  83.75670378   0.         111.54726782
   3.86916704   0.           1.93302295 118.23219416 186.9750624
  25.49762151   4.74910541]
Best fitness (total cost): 898.279250704288

Shipments Matrix:
[[245.  37.  84.]
 [  0. 112.   4.]
 [  0.   2. 118.]]

Inventory Levels:
[187.  25.   5.]
